In [10]:
import pandas as pd
import numpy as np
import scipy.stats as st
import datetime
pd.set_option('display.max_rows', 10)

First, we import the data from the WHO growth table and our measured weights. The datasource is the [WHO](https://www.cdc.gov/growthcharts/who_charts.htm#The%20WHO%20Growth%20Charts)

The table provides the statistical constants we need, but they are only defined on a monthly basis. This isn't sufficient granularity, so we'll need to interpolate the table. I'll make the table daily rather than monthly.

First, however, we will convert the 'Month' column to datetime, starting with the birthdate

In [11]:
# convert to time series with birthdate as start

def construct_baby_growth_table(birthdate,sex='M',characteristic='Weight'):
    if characteristic == 'Weight' and sex =='M':
        file = 'WHO-boys-weight-table.dat'
    elif characteristic == 'Weight' and sex =='F':
        file = 'WHO-girls-weight-table.dat'
    elif characteristic == 'Length' and sex == 'M':
        file = 'WHO-boys-length-table.dat'
    elif characteristic == 'Length' and sex == 'F':
        file = 'WHO-girls-length-table.dat'
    else:
        raise InputError("Please choose 'Weight' or 'Length' for characteristics and 'M' or 'F' for sex")
    data_table = pd.read_csv(file,sep='\t')
    baby_table = pd.DataFrame()

    # Construct a new table starting with the birthdate
    baby_table['Date'] = pd.to_timedelta(data_table['Month'],unit='M') + birthdate
    
    # Copy the stats to the new table
    baby_table[['L','M','S']] = data_table[['L','M','S']]
    
    # Set the date as the index and resample
    baby_table = baby_table.set_index('Date')
    baby_table = baby_table.resample('1D').mean()
    
    # Interpolate the constants
    baby_table = baby_table.interpolate(method='cubic')
    
    return baby_table


Now, we need to interpolate the statistical constants. The default method is linear, but I'm going to use a cubic spline to get nice a nice smooth curve

In [12]:
# the birthdate is Nov-1-2018
birthdate = datetime.datetime(2018,11,1)

elio_weight_table = construct_baby_growth_table(birthdate,sex='M',characteristic='Weight')
elio_weight_table

weights = pd.read_csv('Elio-measured-weight.dat',sep='\t')
weights['Date'] = pd.to_datetime(weights['Date'],format='%d-%b-%Y')

# Computing Z-scores
To compute the z-score of the weight (X), we need the L, M and S parameters:

$$ Z = \frac{(X/M)^L - 1}{LS}, L \neq 0$$

To compute the weight (X), from a Z-score:

$$X = M(1 + LSZ)^{1/L}, L \neq 0$$

Z-scores can be easily translated into percentiles via the normal distribution

In [13]:
def z_score (date,measurement,table):
    X = measurement
    L, M, S = table.loc[date][['L','M','S']]
    Z = ((X/M)**L - 1)/(L*S)
    return Z
    
def percentile (date,meas,table):
    Z = z_score(date,meas,table)
    return st.norm.cdf(Z)

def weight_from_percentile(date,percentile,table):
    L, M, S = table.loc[date][['L','M','S']]
    Z = st.norm.ppf(percentile)
    X = M*(1 + L*S*Z)**(1/L)
    return X

In [14]:
# Compute the percentile
weights['Percentile'] = weights.apply(lambda row: percentile(row['Date'],row['Weight (kg)'],elio_weight_table), axis=1)

## Projection

I want to take Elio's last few measurements and project up to 20 weeks from the last one. Then we can plot his gworth curve against the projection

In [15]:
# Now do the projection 20 weeks from last measurement
projected = pd.DataFrame()
last_meas_date = weights.iloc[-1]['Date']
current_percentile = weights.iloc[-4:]['Percentile'].mean()
# project 20 weeks from last measurement
projected['Date'] = pd.date_range(birthdate,last_meas_date + datetime.timedelta(weeks=20))
projected['Weight (kg)'] = projected.apply(lambda row: weight_from_percentile(row['Date'],current_percentile,elio_weight_table),axis=1)

## Plotting
Finally, we can do the plot of Elio's weights with the projection based on his current percentile

In [16]:
# Plotting
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
import plotly.graph_objs as go
from IPython.display import Image

init_notebook_mode(connected=True)

In [17]:
meas = go.Scatter(x=weights['Date'],
                  y=weights['Weight (kg)'],
                  name='Measured Weights (kg)'
                 )
proj = go.Scatter(x=projected['Date'], 
                  y=projected['Weight (kg)'],
                  name='Projected Weight for {}th Percentile'.format(round(current_percentile*100)),
                  line=dict(dash='dash')
                 )
layout = go.Layout(title="Elio's Weight measured from birth",
                   yaxis = dict(title='Weight (kg)'),
                   xaxis = dict(title='Date')
                  )

In [18]:
fig = go.Figure(data=[meas,proj],layout=layout)
#Image(pio.to_image(fig,width=1200,height=600))

iplot(fig) # interative